In [1]:
import os
import torch

import numpy as np
import pandas as pd

from transformers import (
    AutoModelWithLMHead, 
    AutoTokenizer, 
    Trainer,  
    TrainingArguments,
    DataCollatorForSeq2Seq,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model
from huggingface_hub import login
from tqdm import tqdm

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/ivanevgenyevich/miniforge3/envs/env_tf/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [2]:
login()
np.random.seed(0)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /Users/ivanevgenyevich/.cache/huggingface/token
Login successful


In [3]:
tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-medium')
model = AutoModelWithLMHead.from_pretrained('tinkoff-ai/ruDialoGPT-medium')

def generate(prompt):
    data = tokenizer(prompt, return_tensors='pt')
    output_ids = model.generate(
        **data,
        top_k=50,
        top_p=0.95,
        num_beams=3,
        do_sample=True,
        no_repeat_ngram_size=16,
        temperature=1.2,
        repetition_penalty=1.2,
        length_penalty=1.0,
        eos_token_id=50257,
        max_new_tokens=40
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    output = output[:output.find('@@')]
    return output

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/Users/ivanevgenyevich/miniforge3/envs/env_tf/lib/python3.8/site-packages/transformers/models/auto/modeling_auto.py:1352: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [4]:
lora_config = LoraConfig(
    task_type='CAUSAL_LM', 
    r=64, 
    lora_alpha=16,
    lora_dropout=0.05,
    bias='none',
    target_modules=['c_attn', 'c_proj']
)

model = get_peft_model(model, lora_config)

/Users/ivanevgenyevich/miniforge3/envs/env_tf/lib/python3.8/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [5]:
def make_dialog(dialog):
    flag = 0
    final_dialog = ''
    for message in dialog:
        final_dialog += '@@ВТОРОЙ@@ 'if flag else '@@ПЕРВЫЙ@@ '
        flag = 1 - flag
        final_dialog += message + ' '
    return final_dialog    

def make_dataset(path):
    dataset = pd.read_csv(path)
    dialogs = []
    len_dataset = dataset.shape[0]
    for i in range(len_dataset):
        if dataset.loc[i].isna()['context_1']:
            if i:
                dialogs.append(make_dialog(dialog))
            dialog = []
        if not dataset.loc[i].isna()['response']:
            dialog.append(dataset.loc[i, 'response'])
    return dialogs

In [7]:
def convert(dataset):
    converter_dataset = []
    for i in tqdm(dataset):
        input_ids = tokenizer(dataset)['input_ids'][0]
        converter_dataset.append({'input_ids': input_ids, 'labels': input_ids})
    return converter_dataset

dataset = make_dataset('data/mmro.csv') + make_dataset('data/prac.csv') + make_dataset('data/bayes.csv') + make_dataset('data/nlp.csv')[:300]
np.random.shuffle(dataset)
train_dataset = convert(dataset[:int(0.8 * len(dataset))])
eval_dataset = convert(dataset[int(0.8 * len(dataset)):])

100%|██████████| 171/171 [00:01<00:00, 109.21it/s]


In [8]:
training_arguments = TrainingArguments(
    num_train_epochs=1,
    output_dir='./tg_bot',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=1e-4,
    lr_scheduler_type='cosine',
    warmup_steps=10,
    weight_decay=0.01,
    report_to='wandb',
    evaluation_strategy='steps',
    eval_steps=10,
    save_steps=10, 
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss'
)

training_arguments.set_dataloader(pin_memory=False)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
trainer = Trainer(model=model, args=training_arguments, 
    train_dataset=train_dataset, eval_dataset=eval_dataset, data_collator=data_collator)


In [9]:
trainer.train()
model.save_pretrained(os.path.join('./tg_bot', 'final_checkpoint'))

/Users/ivanevgenyevich/miniforge3/envs/env_tf/lib/python3.8/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: kadchenko-ivan. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/86 [00:00<?, ?it/s]

...

{'eval_loss': 4.245809078216553, 'eval_runtime': 12.3928, 'eval_samples_per_second': 13.798, 'eval_steps_per_second': 1.775, 'epoch': 0.7}
{'loss': 1.7127, 'learning_rate': 1.0542974530180327e-05, 'epoch': 0.81}


  0%|          | 0/22 [00:00<?, ?it/s]

{'eval_loss': 4.214721202850342, 'eval_runtime': 13.0846, 'eval_samples_per_second': 13.069, 'eval_steps_per_second': 1.681, 'epoch': 0.81}
{'loss': 1.5714, 'learning_rate': 1.5299867030334814e-06, 'epoch': 0.93}


  0%|          | 0/22 [00:00<?, ?it/s]

{'eval_loss': 4.207455158233643, 'eval_runtime': 13.016, 'eval_samples_per_second': 13.138, 'eval_steps_per_second': 1.69, 'epoch': 0.93}
{'train_runtime': 193.9409, 'train_samples_per_second': 3.511, 'train_steps_per_second': 0.443, 'train_loss': 3.0131957586421523, 'epoch': 1.0}


In [10]:
model.push_to_hub('my_tg_bot')

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/69.3M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ivankadchenko/my_tg_bot/commit/56d5bdb80c8257fc1ef61f2ad2a791d3387d84b0', commit_message='Upload model', commit_description='', oid='56d5bdb80c8257fc1ef61f2ad2a791d3387d84b0', pr_url=None, pr_revision=None, pr_num=None)